In [1]:
import cv2
from sklearn import svm
from skimage.feature import hog
import os
import random
import numpy as np
import json
from tqdm import tqdm

In [2]:
DATA_TRAIN = 'image_exp/Classification/Data/Train'

train_data = []
categroies = os.listdir(DATA_TRAIN)

# load training data

for category in categroies:
    path = os.path.join(DATA_TRAIN, category)
    print('loading category :' + category)
    for file in os.listdir(path):
        img = cv2.imread(os.path.join(path, file))
        img = cv2.resize(img, (60, 60)) # average size
        fd = hog(img, orientations=9, pixels_per_cell=(6, 6),
                    cells_per_block=(2, 2), multichannel=True)
        train_data.append((fd, category))

random.shuffle(train_data)
print('success!')

loading category :i2
loading category :i4
loading category :i5
loading category :io
loading category :ip
loading category :p11
loading category :p23
loading category :p26
loading category :p5
loading category :pl30
loading category :pl40
loading category :pl5
loading category :pl50
loading category :pl60
loading category :pl80
loading category :pn
loading category :pne
loading category :po
loading category :w57
success!


In [3]:
# divide into train and validation

n = int(0.7 * len(train_data))
train_set = train_data[:n]
val_set = train_data[n:]

print(len(train_set))
print(len(val_set))

10124
4339


In [4]:
# unzip dataset
X_train, Y_train = map(list, zip(*train_set))
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test, Y_test = map(list, zip(*val_set))
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [5]:
classifier = svm.SVC()

In [6]:
classifier.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [7]:
predicted = classifier.predict(X_test)

In [8]:
correct = 0
for i in range(len(X_test)):
    if predicted[i] == Y_test[i]:
        correct += 1

print(correct/len(X_test))

0.9589767227471768


In [9]:
# save model
from sklearn.externals import joblib
joblib.dump(classifier,'exp1.model')

/home/tsinghuaee221/myminiconda/envs/ml/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['exp1.model']

In [10]:
classifier = joblib.load('exp1.model')
DATA_TEST = 'image_exp/Classification/Data/Test'
test_data = []
test_imgs = os.listdir(DATA_TEST)
print('loading test images')
for i in tqdm(range(len(test_imgs))):
    path = os.path.join(DATA_TEST, test_imgs[i])
    img = cv2.imread(path)
    img = cv2.resize(img, (60, 60)) # average size
    fd = hog(img, orientations=9, pixels_per_cell=(6, 6),
                cells_per_block=(2, 2), multichannel=True)
    test_data.append(fd)

  0%|          | 0/3800 [00:00<?, ?it/s]

loading test images


100%|██████████| 3800/3800 [01:25<00:00, 44.23it/s] 


In [11]:
print('testing...')
test_predict = classifier.predict(test_data)

testing...


In [12]:
test_result = {}
for i in tqdm(range(len(test_imgs))):
    test_result[test_imgs[i]] = test_predict[i]

100%|██████████| 3800/3800 [00:00<00:00, 1047335.73it/s]


In [13]:
json.dump(test_result, open('exp1_test.json','w'))